## Proyecto Final de carrera 

### Consigna del Trabajo


#### Elige tu punto de partida entre dos opciones: 

Formula una nueva pregunta, problema o desafío para explorar. Aquí hay una lista no exhaustiva de opciones que puedes elegir:

* **Prueba un modelo de Machine Learning nuevo que no hayas visto en la carrera**. En ese caso, debes explicar su funcionamiento y por qué consideras apropiado aplicarlo. La ganancia no necesariamente tiene que ser en desempeño, también puede ser en interpretabilidad o aplicabilidad.
* **Agrega al proyecto información de otra fuente. ¡Es sumamente válido!** En ese caso, debe estar claramente explicado cómo conseguiste esos datos y qué usos esperas darles. Los datos deben ser accesibles por tu evaluador/a (por ejemplo, puedes subirlos al repositorio). Presta atención a la sensibilidad y privacidad de los datos antes de compartirlos.
* **Continúa el Análisis Exploratorio de Datos**. Recuerda que es tan importante como entrenar un modelo de Machine Learning.
* **Sigue las sugerencias que te propusimos** en los notebooks para continuar el Proyecto 03.

## Checklist de evaluación

* Se evaluará la claridad con la que está expuesto el objetivo a alcanzar y su relevancia.
* Los pasos deben estar correctamente justificados. No deben haber grandes grupos de celdas de código sin explicar su función. Lo mismo aplica para los gráficos. Recuerda que el notebook es un informe.
* Las preguntas que se respondan deben estar correctamente explicitadas.
* Imagina que este proyecto lo usarías para presentar en una entrevista de trabajo, o que lo debes presentar en tu trabajo. * Presta mucha atención a la redacción, presentación de gráficos, etc.
* Debes entregar el proyecto anterior correspondiente (Proyecto 2 si eliges continuar con el dataset de Properati, Proyecto 3 si eliges continuar con alguna aplicación), el cual debe estar subido al repositorio.
* Incorpora las correcciones y/o sugerencias que tu evaluador/a te haya dado en la devolución del proyecto correspondiente anterior.
* Antes de subir tu proyecto a la plataforma Acámica para que sea evaluado, verifica que el Notebook se ejecute sin errores.

## Desarrollo 

Para el desarrollo del informe final de carrera, opte por tomar como punto de partida el Protecto número 3 - Series Temporales,  en donde se desarrolló un modelo de machine learning para predecir el flujo vehícular en la ciudad de buenos aires para el primer trimestre del año 2021. Considero que es posible la mejora del modelo desarrollado intentando responder dos preguntas claves: 
1. La incorporación de un nuevo data set externo al utilizado mejoraría la acertividad del modelo ? Esto surge de la idea de que al vincular información correspondiente a la venta de vehículos vamos a tener una mayor acertividad ya la hipótesis a verificar es que ambas series temporales estan correlacionadas  (Ventas de automóviles vs Flujo Vehicular).
2. Mejorar la predicción mediante el uso de redes neuronales, ya que es un campo de interés personal y abordaría el punto "Probar un modelo de machine learning no visto en la carrera" 

### Recopilaciòn y EDA